This one is for testing some FCNNs

In [ ]:
import pandas as pd
import os, sys
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
import torch

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import fcnn_models

## Data Prep

In [ ]:
def load_csv_data(file_path: str) -> Tuple[np.ndarray, np.array, np.array]:
    """
    Reads the CSV with all the training data: grayscale log-mel spectrogram pixels, label and author of each sample
    And extracts them respectively, resizing the features to fit the CNN input shape (channel, height, width).

    Args:
        file_path (str): Path to the CSV file containing training data.

    Returns:
        Tuple[np.ndarray, np.array, np.array]: Returns features, labels, and authors from the CSV file. Features shape is (N x 70,112),
        while labels and authors are 1D arrays of size N, where N is the number of samples.
    """
    # Load training data
    df = pd.read_csv(file_path)

    print(f"Dataset shape: {df.shape}")
    print(f"Number of classes: {df['label'].nunique()}")
    print(f"Number of authors: {df['author'].nunique()}")

    # Extract labels, authors, and features
    labels = df['label'].values.astype(np.int64)
    authors = df['author'].values
    features = df.drop(columns=['label', 'author']).values.astype(np.float32)
    print(f"Features shape before reshape: {features.shape} (should be N x 70112!)")

    # Convert to 0-1 range
    features /= 255.0

    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)
    print("Authors shape:", authors.shape)
    print("Unique classes:", len(np.unique(labels)))
    print("Unique authors:", len(np.unique(authors)))

    # No need for df variable after extracting features, release memory
    del df

    return features, labels, authors

In [ ]:
# Keep the old CSV-based loading for compatibility (uncomment if needed)
train_data_path = os.path.join('..', 'database', 'meta', 'final', 'train_data.csv')
features, labels, authors = load_csv_data(train_data_path)

# Display class distribution
plt.figure(figsize=(12, 6))
unique_labels, counts = np.unique(labels, return_counts=True)
plt.bar(unique_labels, counts, alpha=0.7)
plt.xlabel('Class ID')
plt.ylabel('Number of Samples')
plt.title('Class Distribution in Training Data')
plt.xticks(unique_labels)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Average samples per class: {len(labels) / len(unique_labels):.1f}")

In [ ]:
# Split with a set seed, las que ya encontre arriba, para tardar menos
from utils.split import get_set_seed_indices, display_split_statistics
seed_single = 245323 # Quality: 0.2671

train_indices, val_indices, best_score, seed = get_set_seed_indices(
    features=features,
    labels=labels, 
    authors=authors,
    test_size=0.2,
    seed=seed_single)

# Extract train and validation data using indices
trFeatures = features[train_indices]
trLabels = labels[train_indices]
trAuthors = authors[train_indices]

vFeatures = features[val_indices]
vLabels = labels[val_indices]
vAuthors = authors[val_indices]

print(f"Training set: {trFeatures.shape[0]} samples")
print(f"Validation set: {vFeatures.shape[0]} samples")
print(f"Split quality score: {best_score}")

display_split_statistics((train_indices, val_indices, best_score, seed), "single")

# Convert labels to one-hot encoding for BirdFCNN_v0 model
num_classes = len(np.unique(labels))

In [ ]:
from utils.fcnn_models import BirdFCNN, BirdFCNN_v0, BirdFCNN_v1, BirdFCNN_v2, BirdFCNN_v3, BirdFCNN_v4, BirdFCNN_v5, BirdFCNN_v6, BirdFCNN_v7, BirdFCNN_v8, BirdFCNN_v9, BirdFCNN_v10, BirdFCNN_v11, BirdFCNN_v12, BirdFCNN_v13, BirdFCNN_v14
from utils.metrics import plot_full_metrics

# Models List - only include models that exist and are complete
models = {
    'BirdFCNN': BirdFCNN,
    'BirdFCNN_v0': BirdFCNN_v0,
    'BirdFCNN_v1': BirdFCNN_v1,
    'BirdFCNN_v2': BirdFCNN_v2,
    'BirdFCNN_v3': BirdFCNN_v3,
    'BirdFCNN_v4': BirdFCNN_v4,
    'BirdFCNN_v5': BirdFCNN_v5,
    'BirdFCNN_v6': BirdFCNN_v6,
    'BirdFCNN_v7': BirdFCNN_v7,
    'BirdFCNN_v8': BirdFCNN_v8,
    'BirdFCNN_v9': BirdFCNN_v9,
    'BirdFCNN_v10': BirdFCNN_v10,
    'BirdFCNN_v11': BirdFCNN_v11,
    'BirdFCNN_v12': BirdFCNN_v12,
    'BirdFCNN_v13': BirdFCNN_v13,
    'BirdFCNN_v14': BirdFCNN_v14
}

In [ ]:
# Quick validation test - ensure all models can be instantiated
print("Validating all models can be instantiated...")
for model_name, model_class in models.items():
    try:
        test_model = model_class(num_classes=num_classes, input_dim=trFeatures.shape[1])
        print(f"✓ {model_name}: OK")
    except Exception as e:
        print(f"✗ {model_name}: ERROR - {str(e)}")

print(f"\nAll models validated. Ready to train with simplified interface:")
print(f"- num_classes: {num_classes}")
print(f"- input_dim: {trFeatures.shape[1]}")
print(f"- train samples: {trFeatures.shape[0]}")
print(f"- val samples: {vFeatures.shape[0]}")

In [ ]:
# Restart kernel and re-import to fix any caching issues
import importlib
import sys
if 'utils.fcnn_models' in sys.modules:
    importlib.reload(sys.modules['utils.fcnn_models'])

# Re-import models
from utils.fcnn_models import BirdFCNN, BirdFCNN_v0, BirdFCNN_v1, BirdFCNN_v2, BirdFCNN_v3, BirdFCNN_v4, BirdFCNN_v5, BirdFCNN_v6, BirdFCNN_v7, BirdFCNN_v8, BirdFCNN_v9, BirdFCNN_v10, BirdFCNN_v11, BirdFCNN_v12, BirdFCNN_v13, BirdFCNN_v14

# Recreate models dictionary
models = {
    'BirdFCNN': BirdFCNN,
    'BirdFCNN_v0': BirdFCNN_v0,
    'BirdFCNN_v1': BirdFCNN_v1,
    'BirdFCNN_v2': BirdFCNN_v2,
    'BirdFCNN_v3': BirdFCNN_v3,
    'BirdFCNN_v4': BirdFCNN_v4,
    'BirdFCNN_v5': BirdFCNN_v5,
    'BirdFCNN_v6': BirdFCNN_v6,
    'BirdFCNN_v7': BirdFCNN_v7,
    'BirdFCNN_v8': BirdFCNN_v8,
    'BirdFCNN_v9': BirdFCNN_v9,
    'BirdFCNN_v10': BirdFCNN_v10,
    'BirdFCNN_v11': BirdFCNN_v11,
    'BirdFCNN_v12': BirdFCNN_v12,
    'BirdFCNN_v13': BirdFCNN_v13,
    'BirdFCNN_v14': BirdFCNN_v14
}

In [ ]:
# Quick test of simplified training interface (just 1 epoch to verify it works)
print("Testing simplified training interface...")
test_model = BirdFCNN_v0(num_classes=num_classes, input_dim=trFeatures.shape[1])

# Test with just 1 epoch to verify the interface works
results = test_model.train_model(
    trainX=trFeatures,
    trainY=trLabels,
    valX=vFeatures,
    valY=vLabels,
    epochs=1  # Just 1 epoch for quick test
)

print("✓ Simplified training interface works!")
print(f"Results keys: {list(results.keys())}")
if 'train_accuracies' in results:
    print(f"Final train accuracy: {results['train_accuracies'][-1]:.4f}")
if 'val_accuracies' in results:
    print(f"Final val accuracy: {results['val_accuracies'][-1]:.4f}")
if 'train_accs' in results:
    print(f"Final train accuracy: {results['train_accs'][-1]:.4f}")
if 'val_accs' in results:
    print(f"Final val accuracy: {results['val_accs'][-1]:.4f}")
print("\nReady for full training with all models!")

## Training

In [ ]:
# Model sweeping - train all models with their individual configurations
import time
import os
from sklearn.metrics import f1_score, confusion_matrix

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

print(f"Using device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

# Quick test: Verify all models can be instantiated
print("🔍 Testing model instantiation...")
for model_name, model_class in models.items():
    try:
        test_model = model_class(num_classes=num_classes, input_dim=trFeatures.shape[1])
        print(f"   ✅ {model_name}: OK")
        del test_model  # Free memory
    except Exception as e:
        print(f"   ❌ {model_name}: {str(e)}")

print(f"\n🔥 Training {len(models)} models with individual configurations")
print("="*70)

# Store results for all models
all_results = {}
saved_models = {}

for model_name, model_class in models.items():
    print(f"\n🔥 Training {model_name}...")
    start_time = time.time()
    
    try:
        # Initialize model
        model = model_class(num_classes=num_classes, input_dim=trFeatures.shape[1])
        
        # Train model - each model handles its own configuration internally
        results = model.train_model(
            trainX=trFeatures,
            trainY=trLabels,
            valX=vFeatures,
            valY=vLabels
        )
        
        # Store results
        all_results[model_name] = results
        
        training_time = time.time() - start_time
        print(f"✅ {model_name} completed in {training_time:.2f} seconds")
        print(f"   Best Val Accuracy: {results['best_val_acc']:.4f}")
        print(f"   Best Val F1: {results['best_val_f1']:.4f}")
        print("-" * 50)
        
    except Exception as e:
        print(f"❌ Error training {model_name}: {str(e)}")
        print("-" * 50)
        continue

print("\n🎉 All models trained successfully!")
print("="*70)

## Results

In [ ]:
# Plot results for all models
print("📊 Plotting results for all models...")
print("="*50)

for model_name, results in all_results.items():
    print(f"Plotting {model_name}...")
    plot_full_metrics(model_name, results['history'], results['confusion_matrix'])

print("\n🎯 Model Performance Summary:")
print("="*50)

# Create summary table
summary_data = []
for model_name, results in all_results.items():
    summary_data.append({
        'Model': model_name,
        'Best Val Accuracy': f"{results['best_val_acc']:.4f}",
        'Best Val F1': f"{results['best_val_f1']:.4f}",
        'Final Train Loss': f"{results['history']['train_losses'][-1]:.4f}",
        'Final Val Loss': f"{results['history']['val_losses'][-1]:.4f}"
    })

# Sort by validation accuracy
summary_data.sort(key=lambda x: float(x['Best Val Accuracy']), reverse=True)

# Display summary table
import pandas as pd
summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print(f"\n🏆 Best model by validation accuracy: {summary_data[0]['Model']}")
print(f"🥈 Second best: {summary_data[1]['Model']}")
print(f"🥉 Third best: {summary_data[2]['Model']}")

# Plot comparative bar chart
plt.figure(figsize=(15, 10))

# Extract metrics for plotting
model_names = [item['Model'] for item in summary_data]
val_accuracies = [float(item['Best Val Accuracy']) for item in summary_data]
val_f1s = [float(item['Best Val F1']) for item in summary_data]

# Subplot 1: Validation Accuracy Comparison
plt.subplot(2, 2, 1)
bars1 = plt.bar(range(len(model_names)), val_accuracies, alpha=0.7, color='skyblue')
plt.xlabel('Models')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy Comparison')
plt.xticks(range(len(model_names)), model_names, rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
# Add value labels on bars
for i, bar in enumerate(bars1):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
             f'{val_accuracies[i]:.3f}', ha='center', va='bottom', fontsize=9)

# Subplot 2: F1 Score Comparison
plt.subplot(2, 2, 2)
bars2 = plt.bar(range(len(model_names)), val_f1s, alpha=0.7, color='lightcoral')
plt.xlabel('Models')
plt.ylabel('Validation F1 Score')
plt.title('Validation F1 Score Comparison')
plt.xticks(range(len(model_names)), model_names, rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
# Add value labels on bars
for i, bar in enumerate(bars2):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
             f'{val_f1s[i]:.3f}', ha='center', va='bottom', fontsize=9)

# Subplot 3: Training vs Validation Loss (final values)
plt.subplot(2, 2, 3)
final_train_losses = [float(item['Final Train Loss']) for item in summary_data]
final_val_losses = [float(item['Final Val Loss']) for item in summary_data]

x = np.arange(len(model_names))
width = 0.35

plt.bar(x - width/2, final_train_losses, width, label='Train Loss', alpha=0.7, color='green')
plt.bar(x + width/2, final_val_losses, width, label='Val Loss', alpha=0.7, color='orange')

plt.xlabel('Models')
plt.ylabel('Final Loss')
plt.title('Final Training vs Validation Loss')
plt.xticks(x, model_names, rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', alpha=0.3)

# Subplot 4: Accuracy vs F1 scatter plot
plt.subplot(2, 2, 4)
plt.scatter(val_accuracies, val_f1s, alpha=0.7, s=100, c='purple')
plt.xlabel('Validation Accuracy')
plt.ylabel('Validation F1 Score')
plt.title('Accuracy vs F1 Score')
plt.grid(True, alpha=0.3)

# Add model name labels to points
for i, model_name in enumerate(model_names):
    plt.annotate(model_name, (val_accuracies[i], val_f1s[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.tight_layout()
plt.show()

print("\n✨ Model sweep completed! Check the plots above for detailed comparisons.")

In [ ]:
# Save the best model and provide recommendations
# The best model is already saved, so we just need to identify it

# Analysis and recommendations based on results
if not all_results:
    print("⚠️ No models were successfully trained!")
else:
    # Find the best model
    best_model_name = summary_data[0]['Model']
    print(f"🏆 Best performing model: {best_model_name}")

    print("\n" + "="*60)
    print("📝 MODEL SWEEP ANALYSIS & RECOMMENDATIONS")
    print("="*60)

    print(f"\n🎯 PERFORMANCE RANKING:")
    for i, item in enumerate(summary_data[:3], 1):
        print(f"{i}. {item['Model']}: Val Acc = {item['Best Val Accuracy']}, Val F1 = {item['Best Val F1']}")

    print(f"\n💡 ARCHITECTURE & TRAINING INSIGHTS:")
    model_configs = {
        'BirdFCNN_v0': "Baseline [512, 256, 128] + SGD + lr=0.01 + batch=64",
        'BirdFCNN': "Standard [512, 128, 32] + Adam + lr=0.001 + batch=32",
        'BirdFCNN_v1': "Residual-inspired with skip connections + AdamW + CosineWarmRestart",
        'BirdFCNN_v2': "Wide network with spectral norm + Adam + lr=0.0005",
        'BirdFCNN_v3': "Deep network [2048->1024->512->256->128] + Adam",
        'BirdFCNN_v4': "Progressive dropout + SGD + momentum",
        'BirdFCNN_v5': "Multi-path ensemble + AdamW + OneCycleLR",
        'BirdFCNN_v6': "Deep narrow with progressive dropout + CosineAnnealingWarmRestarts",
        'BirdFCNN_v7': "Minimal [256] + SGD + lr=0.01",
        'BirdFCNN_v8': "Wide single layer [2048] + Adam + Mish",
        'BirdFCNN_v9': "Balanced 3-layer + AdamW + StepLR",
        'BirdFCNN_v10': "Regularization-heavy + LayerNorm + label_smoothing",
        'BirdFCNN_v11': "Attention-mechanism inspired + CosineAnnealingLR",
        'BirdFCNN_v12': "Knowledge distillation ready + Mish",
        'BirdFCNN_v13': "Complex architecture + various activations",
        'BirdFCNN_v14': "Transformer-inspired + multi-head components"
    }

    for model_name, config in model_configs.items():
        if model_name in [item['Model'] for item in summary_data[:3]]:
            rank = next(i+1 for i, item in enumerate(summary_data) if item['Model'] == model_name)
            print(f"   #{rank} {model_name}: {config}")

    print(f"\n🔬 KEY FINDINGS:")
    print(f"   • Best model achieves {summary_data[0]['Best Val Accuracy']} validation accuracy")
    print(f"   • F1 scores range from {min(val_f1s):.3f} to {max(val_f1s):.3f}")
    best_config = model_configs.get(best_model_name, "Configuration not documented")
    print(f"   • Winning configuration: {best_config}")

    # Calculate overfitting indicators
    print(f"\n⚠️  OVERFITTING ANALYSIS:")
    for item in summary_data[:3]:
        model_name = item['Model']
        train_loss = float(item['Final Train Loss'])
        val_loss = float(item['Final Val Loss'])
        gap = val_loss - train_loss
        print(f"   {model_name}: Train-Val loss gap = {gap:.4f} {'(potential overfitting)' if gap > 0.5 else '(good generalization)'}")

    print(f"\n🚀 NEXT STEPS:")
    print(f"   1. Use {best_model_name} as your primary model")
    print(f"   2. Consider ensemble of top 3 models for better performance")
    print(f"   3. The winning configuration uses: {best_config}")
    print(f"   4. Experiment with data augmentation techniques")
    print(f"   5. Try transfer learning with pre-trained audio models")
    print(f"   6. Consider hyperparameter tuning for the best performing architectures")

    print("="*60)